In [54]:
import os

import cv2 as cv2
import numpy as np
import open3d as o3d

In [55]:
print(o3d.__version__)

0.10.0.0


In [56]:
def load_point_clouds():
    pcds = []
    for i in range(2):
        pcd = o3d.io.read_point_cloud(f"./camera moving/D1/p{i+1}.pcd")
        pcds.append(pcd)
    return pcds

In [57]:
pcds_down = load_point_clouds()
o3d.visualization.draw_geometries(pcds_down)

In [76]:
def draw_registration_result_original_color(source, target, transformation):
    source_temp = source
    source_temp.transform(transformation)

def pairwise_registration(source, target, max_correspondence_distance_coarse, max_correspondence_distance_fine):
    print("Apply point-to-plane ICP")
    icp_coarse = o3d.registration.registration_icp(
        source, target, max_correspondence_distance_coarse, np.identity(4),
        o3d.registration.TransformationEstimationPointToPlane(),
        o3d.registration.ICPConvergenceCriteria(relative_fitness=1e-6,
                                                          relative_rmse=1e-6,
                                                          max_iteration=50))
    icp_fine = o3d.registration.registration_icp(
        source, target, max_correspondence_distance_fine,
        icp_coarse.transformation,
        o3d.registration.TransformationEstimationPointToPlane(),
        o3d.registration.ICPConvergenceCriteria(relative_fitness=1e-6,
                                                          relative_rmse=1e-6,
                                                          max_iteration=50))
    transformation_icp = icp_fine.transformation
    information_icp = o3d.registration.get_information_matrix_from_point_clouds(
        source, target, max_correspondence_distance_fine,
        icp_fine.transformation)
    return transformation_icp, information_icp


def full_registration(pcds, max_correspondence_distance_coarse,
                      max_correspondence_distance_fine):
    
    pose_graph = o3d.registration.PoseGraph()
    odometry = np.identity(4)
    pose_graph.nodes.append(o3d.registration.PoseGraphNode(odometry))
    n_pcds = len(pcds)
    for source_id in range(n_pcds):
        for target_id in range(source_id + 1, n_pcds):
            
            source = pcds[source_id]
            target = pcds[target_id]

            current_transformation = np.identity(4)
            draw_registration_result_original_color(source, target, current_transformation)

            source.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.01 * 2, max_nn=30))
            target.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.01 * 2, max_nn=30))


            # estimate_normals(source, search_param = KDTreeSearchParamHybrid( radius = 0.1, max_nn = 30))
            # estimate_normals(target, search_param = KDTreeSearchParamHybrid( radius = 0.1, max_nn = 30))

            transformation_icp, information_icp = pairwise_registration(
                source,target, max_correspondence_distance_coarse, max_correspondence_distance_fine)
            print("Build o3d.registration.PoseGraph")
            if target_id == source_id + 1:  # odometry case
                odometry = np.dot(transformation_icp, odometry)
                pose_graph.nodes.append(
                    o3d.registration.PoseGraphNode(
                        np.linalg.inv(odometry)))
                pose_graph.edges.append(
                    o3d.registration.PoseGraphEdge(source_id,
                                                             target_id,
                                                             transformation_icp,
                                                             information_icp,
                                                             uncertain=False))
            else:  # loop closure case
                pose_graph.edges.append(
                    o3d.registration.PoseGraphEdge(source_id,   
                                                             target_id,
                                                             transformation_icp,
                                                             information_icp,
                                                             uncertain=True))
    return pose_graph

In [77]:
print("Full registration ...")
voxel_size = 0.001
max_correspondence_distance_coarse = voxel_size * 100
max_correspondence_distance_fine = voxel_size * 1.5
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    pose_graph = full_registration(pcds_down,
                                   max_correspondence_distance_coarse,
                                   max_correspondence_distance_fine)

Full registration ...
Apply point-to-plane ICP
[Open3D DEBUG] ICP Iteration #0: Fitness 0.9578, RMSE 0.0264
[Open3D DEBUG] Residual : 1.87e-04 (# of elements : 526758)
[Open3D DEBUG] ICP Iteration #1: Fitness 0.9478, RMSE 0.0267
[Open3D DEBUG] Residual : 1.48e-04 (# of elements : 521264)
[Open3D DEBUG] ICP Iteration #2: Fitness 0.9451, RMSE 0.0269
[Open3D DEBUG] Residual : 1.46e-04 (# of elements : 519756)
[Open3D DEBUG] ICP Iteration #3: Fitness 0.9430, RMSE 0.0270
[Open3D DEBUG] Residual : 1.44e-04 (# of elements : 518631)
[Open3D DEBUG] ICP Iteration #4: Fitness 0.9416, RMSE 0.0270
[Open3D DEBUG] Residual : 1.43e-04 (# of elements : 517861)
[Open3D DEBUG] ICP Iteration #5: Fitness 0.9407, RMSE 0.0270
[Open3D DEBUG] Residual : 1.43e-04 (# of elements : 517367)
[Open3D DEBUG] ICP Iteration #6: Fitness 0.9402, RMSE 0.0270
[Open3D DEBUG] Residual : 1.42e-04 (# of elements : 517081)
[Open3D DEBUG] ICP Iteration #7: Fitness 0.9399, RMSE 0.0270
[Open3D DEBUG] Residual : 1.42e-04 (# of elem

In [78]:
print("Optimizing PoseGraph ...")
option = o3d.registration.GlobalOptimizationOption(
    max_correspondence_distance=max_correspondence_distance_fine,
    edge_prune_threshold=0.25,
    reference_node=0)
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    o3d.registration.global_optimization(
        pose_graph,
        o3d.registration.GlobalOptimizationLevenbergMarquardt(),
        o3d.registration.GlobalOptimizationConvergenceCriteria(),
        option)

Optimizing PoseGraph ...
[Open3D DEBUG] Validating PoseGraph - finished.
[Open3D DEBUG] [GlobalOptimizationLM] Optimizing PoseGraph having 2 nodes and 1 edges.
[Open3D DEBUG] Line process weight : 0.089237
[Open3D DEBUG] [Initial     ] residual : 4.066775e-31, lambda : 3.966100e-01
[Open3D DEBUG] Maximum coefficient of right term < 1.000000e-06
[Open3D DEBUG] [GlobalOptimizationLM] Optimizing PoseGraph having 2 nodes and 1 edges.
[Open3D DEBUG] Line process weight : 0.089237
[Open3D DEBUG] [Initial     ] residual : 4.066775e-31, lambda : 3.966100e-01
[Open3D DEBUG] Maximum coefficient of right term < 1.000000e-06
[Open3D DEBUG] CompensateReferencePoseGraphNode : reference : 0


In [79]:
print("Transform points and display")
for point_id in range(len(pcds_down)):
    print(pose_graph.nodes[point_id].pose)
    pcds_down[point_id].transform(pose_graph.nodes[point_id].pose)
o3d.visualization.draw_geometries(pcds_down)


Transform points and display
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[ 0.9997812  -0.01925465  0.0081738   0.00203791]
 [ 0.01878108  0.998336    0.05452057  0.01410176]
 [-0.00920998 -0.05435513  0.99847919  0.00395994]
 [ 0.          0.          0.          1.        ]]


RuntimeError: [Open3D ERROR] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 

In [80]:
pcds_down

[geometry::PointCloud with 549970 points.,
 geometry::PointCloud with 584370 points.]

In [81]:
pcds = pcds_down
pcd_combined = o3d.geometry.PointCloud()
for point_id in range(len(pcds)):
    pcds[point_id].transform(pose_graph.nodes[point_id].pose)
    pcd_combined += pcds[point_id]
# pcd_combined_down = pcd_combined.voxel_down_sample(voxel_size=voxel_size)
o3d.io.write_point_cloud("multiway_registration.ply", pcd_combined)
o3d.visualization.draw_geometries([pcd_combined])